In [1]:
import VariantCalling as vc
import matplotlib.pyplot as plt
import pickle
import numpy as np
import os

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers.legacy import Adam

In [3]:
files = ["data_0_1.pkl", "data_0_5.pkl", "pickle/simulated/data_01_5.pkl", "data_03_5.pkl", "data_05_5.pkl", "data_1_5.pkl"]

In [16]:
def create_cnn_model(input_shape):
    input_layer = Input(shape=input_shape)

    # Encoder
    encoded = Conv2D(64, (8, 8), activation='relu', padding='same')(input_layer)
    encoded = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)

    # Decoder
    decoded = Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(encoded)
    decoded = Conv2DTranspose(64, (8, 8), activation='relu', padding='same')(decoded)
    decoded = Conv2D(1, (8, 8), activation='sigmoid', padding='same')(decoded)

    model = Model(input_layer, decoded)

    return model

In [17]:
def load_data(file_name):
    img_row = 100
    img_col = 178
    with open("pickle/simulated/data_01_5.pkl", "rb") as file:
        noisy, clean = pickle.load(file)
    
    noisy = noisy.astype('float32') / 3
    clean = clean.astype('float32') / 3
    noisy = noisy.reshape(noisy.shape[0], img_row, img_col, 1)
    clean = clean.reshape(clean.shape[0], img_row, img_col, 1)
    return noisy, clean

In [18]:
input_shape = (100, 178, 1)

model1 = create_cnn_model(input_shape)
model2 = create_cnn_model(input_shape)

learning_rate = 0.0005
model1.compile(optimizer=Adam(learning_rate=learning_rate), loss=MeanSquaredError())
print(model1.summary())

learning_rate = 0.0005
model2.compile(optimizer=Adam(learning_rate=learning_rate), loss=MeanSquaredError())
print(model2.summary())


Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 100, 178, 1)]     0         
                                                                 
 conv2d_18 (Conv2D)          (None, 100, 178, 64)      4160      
                                                                 
 conv2d_19 (Conv2D)          (None, 100, 178, 128)     73856     
                                                                 
 conv2d_transpose_12 (Conv2  (None, 100, 178, 128)     147584    
 DTranspose)                                                     
                                                                 
 conv2d_transpose_13 (Conv2  (None, 100, 178, 64)      524352    
 DTranspose)                                                     
                                                                 
 conv2d_20 (Conv2D)          (None, 100, 178, 1)       4097

In [19]:
# Curriculum
batch_size = 32
img_row = 100
img_col = 178

# Part 1
noisy, clean = load_data("pickle/simulated/data_0_1.pkl")

epochs = 10
model1.fit(noisy, clean, batch_size=batch_size, epochs=epochs, shuffle=True)

denoised_image1 = model1.predict(noisy)
denoised_image1_quantised = np.round(denoised_image1 * 3) / 3

epochs = 4
model2.fit(denoised_image1_quantised, clean, batch_size=batch_size, epochs=epochs, shuffle=True)

# Part 2
noisy, clean = load_data("pickle/simulated/data_0_5.pkl")

epochs = 5
model1.fit(noisy, clean, batch_size=batch_size, epochs=epochs, shuffle=True)

denoised_image1 = model1.predict(noisy)
denoised_image1_quantised = np.round(denoised_image1 * 3) / 3

epochs = 3
model2.fit(denoised_image1_quantised, clean, batch_size=batch_size, epochs=epochs, shuffle=True)

# Part 3
noisy, clean = load_data("pickle/simulated/data_01_5.pkl")

epochs = 7
model1.fit(noisy, clean, batch_size=batch_size, epochs=epochs, shuffle=True)

denoised_image1 = model1.predict(noisy)
denoised_image1_quantised = np.round(denoised_image1 * 3) / 3

epochs = 2
model2.fit(denoised_image1_quantised, clean, batch_size=batch_size, epochs=epochs, shuffle=True)

# Part 4
noisy, clean = load_data("pickle/simulated/data_03_5.pkl")

epochs = 6
model1.fit(noisy, clean, batch_size=batch_size, epochs=epochs, shuffle=True)

denoised_image1 = model1.predict(noisy)
denoised_image1_quantised = np.round(denoised_image1 * 3) / 3

epochs = 2
model2.fit(denoised_image1_quantised, clean, batch_size=batch_size, epochs=epochs, shuffle=True)

# Part 5
noisy, clean = load_data("pickle/simulated/data_05_5.pkl")

epochs = 5
model1.fit(noisy, clean, batch_size=batch_size, epochs=epochs, shuffle=True)

denoised_image1 = model1.predict(noisy)
denoised_image1_quantised = np.round(denoised_image1 * 3) / 3

epochs = 3
model2.fit(denoised_image1_quantised, clean, batch_size=batch_size, epochs=epochs, shuffle=True)

# Part 6
noisy, clean = load_data("pickle/simulated/data_1_5.pkl")

epochs = 7
model1.fit(noisy, clean, batch_size=batch_size, epochs=epochs, shuffle=True)

denoised_image1 = model1.predict(noisy)
denoised_image1_quantised = np.round(denoised_image1 * 3) / 3

epochs = 3
model2.fit(denoised_image1_quantised, clean, batch_size=batch_size, epochs=epochs, shuffle=True)

model1.save('model1_full')
model2.save('model2_full')

Epoch 1/10


2023-08-06 02:52:32.406621: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 775s 826ms/step - loss: 0.0039
Epoch 2/10
938/938 [==============================] - 766s 816ms/step - loss: 7.2002e-04
Epoch 3/10
938/938 [==============================] - 765s 816ms/step - loss: 4.7566e-04
Epoch 4/10
938/938 [==============================] - 765s 816ms/step - loss: 3.5410e-04
Epoch 5/10
938/938 [==============================] - 765s 816ms/step - loss: 2.7206e-04
Epoch 6/10
938/938 [==============================] - 765s 816ms/step - loss: 2.1770e-04
Epoch 7/10
938/938 [==============================] - 765s 816ms/step - loss: 1.7383e-04
Epoch 8/10
938/938 [==============================] - 765s 816ms/step - loss: 1.5007e-04
Epoch 9/10
938/938 [==============================] - 765s 816ms/step - loss: 1.3488e-04
Epoch 10/10
  1/938 [..............................] - ETA: 6:51

2023-08-06 05:00:21.448104: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 303s 323ms/step
Epoch 1/4


2023-08-06 05:05:39.436807: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 766s 816ms/step - loss: 0.0021
Epoch 2/4
938/938 [==============================] - 765s 816ms/step - loss: 2.0433e-04
Epoch 3/4
938/938 [==============================] - 765s 816ms/step - loss: 1.6961e-04
Epoch 4/4
938/938 [==============================] - 765s 816ms/step - loss: 1.6207e-04
Epoch 1/5
938/938 [==============================] - 765s 816ms/step - loss: 1.1279e-04
Epoch 2/5
938/938 [==============================] - 765s 816ms/step - loss: 1.0387e-04
Epoch 3/5
938/938 [==============================] - 765s 816ms/step - loss: 9.7640e-05
Epoch 4/5
938/938 [==============================] - 765s 816ms/step - loss: 9.3009e-05
Epoch 5/5
938/938 [==============================] - 303s 324ms/step
Epoch 1/3
938/938 [==============================] - 765s 816ms/step - loss: 1.2869e-04
Epoch 2/3
938/938 [==============================] - 765s 816ms/step - loss: 1.2438e-04
Epoch 3/3
938/938 [==============================] - 765s 816ms/s

: 

: 

In [ ]:
model1